# Define and Train Network

## Set Environment

In [23]:
import numpy as np
import tensorflow as tf
from time import time
import math
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import layers

(train_x, train_y), (test_x, test_y) = cifar10.load_data()

## Normalize Dataset

In [13]:
# Code generated by Google Search term: "keras normalize image"

norm_layer = Normalization()
norm_layer.adapt(train_x)  # Calculate mean and variance from training data

# model = Sequential([
#     norm_layer,
#     # ... rest of your model layers
# ])

## Define CNN

### CNN 1

In [24]:
# Code hints from https://www.tensorflow.org/tutorials/images/cnn

model = Sequential()
model.add(norm_layer)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

### CNN 2

In [27]:
# Initial model suggestions from DeepNote AI auto-filler

model = Sequential([
    norm_layer,
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

## Train Model

In [ ]:
# https://www.tensorflow.org/tutorials/images/cnn

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e480598-2102-4b3c-a7af-02aec7eef77c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>